<a href="https://colab.research.google.com/github/nfilipas/handson-ml3/blob/main/exercises/chapter7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 8

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV

In [2]:
mnist = fetch_openml("mnist_784", as_frame=False)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
X = mnist.data
y = mnist.target

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, random_state=42, stratify=y, test_size=10000)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, random_state=43, stratify=y_train_val,
    test_size=10000)

In [13]:
forest_clf = RandomForestClassifier(random_state=42, n_jobs=-1)
ext_tree_clf = ExtraTreeClassifier(random_state=42)
svm_clf = LinearSVC(max_iter=100, tol=20)

clfs = [forest_clf, ext_tree_clf, svm_clf]
pipelines = []

for clf in clfs:
    clf_pipeline = make_pipeline(StandardScaler(), clf)
    clf_pipeline.fit(X_train, y_train)
    pipelines.append(clf_pipeline)

In [14]:
for pipeline in pipelines:
    print(f"{pipeline[1]}: {accuracy_score(y_val, pipeline.predict(X_val))}")

RandomForestClassifier(n_jobs=-1, random_state=42): 0.9691
ExtraTreeClassifier(random_state=42): 0.8157
LinearSVC(max_iter=100, tol=20): 0.8625


In [15]:
hard_vot_clf = VotingClassifier(estimators=[("forest_clf", forest_clf),
                                            ("ext_tree_clf", ext_tree_clf),
                                            ("calib_svm_clf", svm_clf)])
soft_vot_clf = VotingClassifier(estimators=[("forest_clf", forest_clf),
                                            ("ext_tree_clf", ext_tree_clf)],
                                voting="soft")

hard_vot_clf.fit(X_train, y_train)
soft_vot_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('forest_clf',
                              RandomForestClassifier(n_jobs=-1,
                                                     random_state=42)),
                             ('ext_tree_clf',
                              ExtraTreeClassifier(random_state=42))],
                 voting='soft')

In [16]:
print(f"Hard voting classifier: accuracy = {accuracy_score(y_val, hard_vot_clf.predict(X_val))}")
print(f"Soft voting classifier: accuracy = {accuracy_score(y_val, soft_vot_clf.predict(X_val))}")

Hard voting classifier: accuracy = 0.9367
Soft voting classifier: accuracy = 0.8164


In [ ]:
for pipeline in pipelines:
    print(f"{pipeline[1]}: {accuracy_score(y_test, pipeline.predict(X_test))}")
